In [1]:
import pandas as pd
import numpy as np
from joblib import Parallel, delayed
from datetime import datetime 
import xlsxwriter
import os, shutil

import time

In [2]:
#  parameters
circle="Zone"
month="nov2023"
month_doc=9
month_unpaid=11
date='22112023'
ftpHost = 'ftp.uppclonline.com'
ftpUname = 'mvftpreport'
ftpPass = 'Mvftp@321'
localFolderPath = f".\\{month}\\ots"
remoteFolder = "OTS_NOV2023/OTS_DATA/ELIGIBLE_LIST"  
# ftp://mvftpreport@ftp.uppclonline.com/OTS_NOV2023/OTS_DATA/ELIGIBLE_LIST/


In [3]:
def bifurcation(column_name,area):
    #area = [Circle,Division]
# column_name = 'Country'
# replace_symbols = ['>', '<', ':', '"', '/', '\\\\', '\|', '\?', '\*']
# df[column_name] = (
#     df[column_name].replace(replace_symbols, '', regex=True).str.strip().str.title()
# )

    os.mkdir(f'{month}/{circle}/master_{area}') 
    unique_values = df[column_name].unique()
    print(unique_values)
    for unique_value in unique_values:
        start = time.time()
        print(unique_value)
        df_output = df[df[column_name].str.contains(unique_value)]
        output_path = os.path.join(f'{month}/{circle}/master_{area}', str(unique_value) + '.csv')
        df_output.to_csv(output_path, index=False)
        end = time.time()
        print("CSV:", end - start)

In [4]:
import ftplib
import os
import gzip
import shutil
import time



def downloadFilesFromFtp(localfolderPath, targetFilenames, ftpHost, ftpUname, ftpPass, remoteWorkingDirectory):
    # initialize the flag that specifies if download is success
    isDownloadSuccess: bool = False

    # create an FTP client instance, use the timeout parameter for slow connections only
    ftp = ftplib.FTP(timeout=30)
    ftp.encoding = "utf-8"

    # connect to the FTP server
    ftp.connect(ftpHost)

    # login to the FTP server
    ftp.login(ftpUname, ftpPass)

    # change current working directory if specified
    if not (remoteWorkingDirectory == None or remoteWorkingDirectory.strip() == ""):
        _ = ftp.cwd(remoteWorkingDirectory)

    # iterate through each remote filename and download
    for fItr in range(len(targetFilenames)):
        targetFilename = targetFilenames[fItr]
        print(targetFilename)
        # derive the local file path by appending the local folder path with remote filename
        localFilePath = os.path.join(localfolderPath, targetFilename)
        print(localFilePath)
        print("downloading file {0}".format(targetFilename))
        # download FTP file using retrbinary function
        with open(localFilePath, "wb") as file:
            retCode = ftp.retrbinary("RETR " + targetFilename, file.write)
         
        # with gzip.open(localFilePath, 'rb') as f_in:
        #     with open(localFilePath[:-3], 'wb') as f_out:
        #         shutil.copyfileobj(f_in, f_out)
        # os.remove(localFilePath)
    # read file
            
    # send QUIT command to the FTP server and close the connection
    ftp.quit()

    # check if download is success using the return code (retCode)
    if retCode.startswith('226'):
        isDownloadSuccess = True
    return isDownloadSuccess



remoteFilenames = [ 'OTS_LIST1_2023.csv','OTS_LIST2_2023.csv','OTS_LIST3_2023.csv','OTS_LIST4_2023.csv','OTS_LIST5_2023.csv','OTS_LIST6_2023.csv','OTS_LIST7_2023.csv','OTS_LIST8_2023.csv']


# run the function to download the files from FTP server
isDownloadSuccess = downloadFilesFromFtp(
    localFolderPath,remoteFilenames, ftpHost,ftpUname, ftpPass, remoteFolder)

print("download status = {0}".format(isDownloadSuccess))

error_perm: 550 Failed to change directory.

In [9]:
bill="ots"

In [10]:
def loop(file_name):
    return pd.read_csv(f"{month}/{bill}/{file_name}",encoding='windows-1252')

In [11]:
mylist = os.listdir(f"{month}/{bill}")

In [12]:
mylist

[]

In [7]:
start = time.time()
df = []
df = Parallel(n_jobs=-1, verbose=10,prefer="threads")(delayed(loop)(file_name) for file_name in mylist)
df = pd.concat(df, ignore_index=True)

end = time.time()
print("CSV:", end - start)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
C:\Users\HP\AppData\Local\Temp\ipykernel_22400\994463276.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(f"{month}/{bill}/{file_name}",encoding='windows-1252')
C:\Users\HP\AppData\Local\Temp\ipykernel_22400\994463276.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(f"{month}/{bill}/{file_name}",encoding='windows-1252')
[Parallel(n_jobs=-1)]: Done   2 out of   8 | elapsed:   24.2s remaining:  1.2min
C:\Users\HP\AppData\Local\Temp\ipykernel_22400\994463276.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(f"{month}/{bill}/{file_name}",encoding='windows-1252')
[Parallel(n_jobs=-1)]: Done   3 out of   8 | elapsed:   26.0s remaining:   43.4s
C:\Users\HP\AppData\Local\Temp\ipykerne

CSV: 32.46273851394653


In [8]:
df = df[(df['ZONE_NAME']=='DEVIPATAN')]

In [ ]:
df.info()

In [9]:
df['OTS_TYP'].value_counts()

TYP1A    1062507
TYP1B      65559
TYP2A      27313
TYP5        7076
TYP2B       2465
TYP6         739
TYP4          89
Name: OTS_TYP, dtype: int64

In [10]:
df['OTS_TYP'] = df['OTS_TYP'].astype('category')
df['OTS_TYP'] = df['OTS_TYP'].cat.rename_categories({'TYP1B': 'LMV-1 (more than 1 kW)', 'TYP1A': 'LMV-1 (upto 1 kW)', 'TYP2B':'LMV-2 (more than 3 Kw)', 'TYP2A': 'LMV-2 (upto 3 kW)', 'TYP4':'LMV-4B', 'TYP5':'LMV-5', 'TYP6': 'LMV-6'})

In [11]:
ots_1=df.pivot_table(index=['CIRCLE_NAME','DIV_NAME'], columns=['OTS_TYP'],values=['ACCT_ID','TOTAL_AMT_PAYABLE_31OCT'],
                    aggfunc={'ACCT_ID':'count','TOTAL_AMT_PAYABLE_31OCT':np.sum},fill_value=0,margins=True,margins_name='Total' )


In [13]:
df.to_excel(f"{month}/{circle}_cat_oct_eligible.xlsx")

ValueError: This sheet is too large! Your sheet size is: 1165753, 22 Max sheet size is: 1048576, 16384

In [14]:
df['OTS_TYP'].value_counts()

LMV-1 (upto 1 kW)         1062507
LMV-1 (more than 1 kW)      65559
LMV-2 (upto 3 kW)           27313
LMV-5                        7076
LMV-2 (more than 3 Kw)       2465
LMV-6                         739
LMV-4B                         89
Name: OTS_TYP, dtype: int64

In [15]:
# df = df[df.OTS_TYP.isin(['LMV-2 (more than 3 Kw)', 'LMV-4B', 'LMV-5', 'LMV-6'])]
df = df[df["TOTAL_AMT_PAYABLE_31OCT"]>=100000]

In [21]:
df.to_excel(f"{month}/{circle}_cat_oct_eligible1lac.xlsx")

In [16]:
df['OTS_TYP'].value_counts()

LMV-1 (upto 1 kW)         28480
LMV-1 (more than 1 kW)    12507
LMV-2 (upto 3 kW)          2970
LMV-5                      1081
LMV-2 (more than 3 Kw)      447
LMV-6                       157
LMV-4B                        6
Name: OTS_TYP, dtype: int64

In [19]:
bifurcation("CIRCLE_NAME",'Circle1lac') 

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'oct2023/Zone/master_Circle'

In [70]:

# # names = df['OTS_TYP'].unique().tolist()
# mylist = os.listdir(f"{month}/zone/master_DIV_NAME_3")
# for file in mylist:
#     writer = pandas.ExcelWriter("MyData.xlsx", engine='xlsxwriter')

# for myname in names:
#     mydf = df.loc[df.name==myname]
#     mydf.to_excel(writer, sheet_name=myname)

# writer.save()

['EDD BALRAMPUR.csv',
 'EDD I BAHRAICH.csv',
 'EDD I GONDA.csv',
 'EDD II BAHRAICH NANPARA.csv',
 'EDD II GONDA.csv',
 'EDD III BAHRAICH KAISERGANJ.csv',
 'EDD III GONDA COLONELGANJ.csv',
 'EDD IV GONDA MANAKPUR.csv',
 'EDD SHARVASTI.csv',
 'EDD TULSIPUR.csv']

In [77]:
import os

directory_path = "oct2023/zone/master_DIV_NAME_3"
directory_stat = os.stat(directory_path)
uid = directory_stat.st_uid


In [79]:
current_user = os.getuser()
os.chown(directory_path, current_user, current_user)

AttributeError: module 'os' has no attribute 'getuser'

In [80]:
os.chmod(directory_path, 0o700)

In [24]:
df.to_excel(f"{month}/LESA_CIS.xlsx")

In [86]:
mylist = os.listdir(f"{month}/zone/master_DIV_NAME_3")
for file in mylist:
    print(file)
    df = pd.read_csv(f"{month}/zone/master_DIV_NAME_3/{file}")

EDD BALRAMPUR.csv
EDD I BAHRAICH.csv
EDD I GONDA.csv
EDD II BAHRAICH NANPARA.csv
EDD II GONDA.csv
EDD III BAHRAICH KAISERGANJ.csv
EDD III GONDA COLONELGANJ.csv
EDD IV GONDA MANAKPUR.csv
EDD SHARVASTI.csv
EDD TULSIPUR.csv
new


PermissionError: [Errno 13] Permission denied: 'oct2023/zone/master_DIV_NAME_3/new'

In [87]:
import pandas as pd

# Read the Excel sheet into a Pandas DataFrame
mylist = os.listdir(f"{month}/zone/master_DIV_NAME_3")
for file in mylist:
    df = pd.read_csv(f"{month}/zone/master_DIV_NAME_3/{file}")
    
    writer = pd.ExcelWriter(f"{month}/new/{file[:-3]}.xlsx",engine = 'xlsxwriter')
    for cat in df['OTS_TYP'].unique():
        newdf = df[df['OTS_TYP'] == cat]
        newdf.to_excel(writer,sheet_name = cat,index = False)
        
    writer.save()

# Split the DataFrame into multiple DataFrames based on the 'Country' column value
#     df_split = df.groupby('OTS_TYP')

# # Create a new Excel sheet for each DataFrame
#     for name, group in df_split:
#         group.to_excel(f'{month}/zone/master_DIV_NAME_3/new/{name}.xlsx', index=False)

C:\Users\HP\AppData\Local\Temp\ipykernel_21652\306236240.py:13: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
C:\Users\HP\AppData\Local\Temp\ipykernel_21652\306236240.py:13: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
C:\Users\HP\AppData\Local\Temp\ipykernel_21652\306236240.py:13: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
C:\Users\HP\AppData\Local\Temp\ipykernel_21652\306236240.py:13: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
C:\Users\HP\AppData\Local\Temp\ipykernel_21652\306236240.py:13: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future